In [628]:
import pandas as pd 
import numpy as np
from numpy import array
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
from ipynb.fs.full.evaluation import *
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from ipynb.fs.full.Data_Processing import *
from sklearn import preprocessing
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
torch.set_printoptions(linewidth=120) #Display options for output
torch.set_grad_enabled(True) # Already on by default
torch.manual_seed(0)
from torch_lr_finder import LRFinder
import pickle
import torch.utils.data as data_utils
from collections import namedtuple
import time
from sklearn.model_selection import KFold

### CNN

In [629]:
class EEGNet(nn.Module):
    def __init__(self, num_classes, combined):
        super(EEGNet, self).__init__()
        self.T = 120
        self.combined = combined
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 8), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(4*2*7, num_classes)
        
    def forward(self, x):
        # Layer 1
        x = x.float()
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.1)
        x = x.permute(0, 3, 1, 2)

        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.1)
        x = self.pooling2(x)

        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.1)
        x = self.pooling3(x)

        # FC Layer
        x = x.view(-1, 4*2*7)
        if self.combined == False:       
            x = self.fc1(x)
        return x
    

    
    # for 60 timepoints = 4*2*4 and -1
    # 120 timepoints = 4* 2* 7 and -1
    # https://discuss.pytorch.org/t/runtimeerror-shape-1-400-is-invalid-for-input-of-size/33354
    # https://discuss.pytorch.org/t/valueerror-expected-input-batch-size-324-to-match-target-batch-size-4/24498/2

### CNN + RNN

https://ai.stackexchange.com/questions/3156/how-to-select-number-of-hidden-layers-and-number-of-memory-cells-in-an-lstm

In [630]:
# class CNN(nn.Module):
#     def __init__(self, num_classes):
#         super(CNN, self).__init__()
#         self.T = 120
        
#         # Layer 1
#         self.conv1 = nn.Conv2d(1, 16, (1, 8), padding = 0)
#         self.batchnorm1 = nn.BatchNorm2d(16, False)
        
#         # Layer 2
#         self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
#         self.conv2 = nn.Conv2d(1, 4, (2, 32))
#         self.batchnorm2 = nn.BatchNorm2d(4, False)
#         self.pooling2 = nn.MaxPool2d(2, 4)
        
#         # Layer 3
#         self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
#         self.conv3 = nn.Conv2d(4, 4, (8, 4))
#         self.batchnorm3 = nn.BatchNorm2d(4, False)
#         self.pooling3 = nn.MaxPool2d((2, 4))
        
#         # FC Layer
#         # NOTE: This dimension will depend on the number of timestamps per sample in your data.
#         # I have 120 timepoints. 
#         self.fc1 = nn.Linear(4*2*7, num_classes)
        
#     def forward(self, x):
#         # Layer 1
#         x = x.float()
#         x = F.elu(self.conv1(x))
#         x = self.batchnorm1(x)
#         x = F.dropout(x, 0.1)
#         x = x.permute(0, 3, 1, 2)

#         # Layer 2
#         x = self.padding1(x)
#         x = F.elu(self.conv2(x))
#         x = self.batchnorm2(x)
#         x = F.dropout(x, 0.1)
#         x = self.pooling2(x)

#         # Layer 3
#         x = self.padding2(x)
#         x = F.elu(self.conv3(x))
#         x = self.batchnorm3(x)
#         x = F.dropout(x, 0.1)
#         x = self.pooling3(x)

#         # FC Layer
#         x = x.view(-1, 4*2*7)
# # #         x = self.fc1(x)
#         return x

#     def predict(self, loader):
#         predictions = []
#         with torch.no_grad():
#             for data in loader:
#                 inputs = data[0].cuda(0)
#                 labels = data[1].cuda(0)
#                 outputs = net(inputs)
#                 _, predicted = torch.max(outputs.data, 1)
#                 predictions.append(predicted)
#                 total += labels.size(0)
#                 correct += (predicted == labels).sum().item()
#         acc = (correct/total * 100)
#         return predictions
    

class Combine(nn.Module):
    def __init__(self, num_classes, combined):
        super(Combine, self).__init__()
        self.cnn = EEGNet(num_classes,combined)
        self.rnn = nn.LSTM(
            input_size=56, 
            hidden_size=16, 
            num_layers=1,
            batch_first=True)
        self.linear = nn.Linear(16 ,num_classes)

    def forward(self, x):
        batch_size, C, timepoints, channels = x.size()
        c_in = x
        c_out = self.cnn(c_in)
        r_in = c_out.view(batch_size, -1 , c_out.shape[1])
        r_out, (h_n, h_c) = self.rnn(r_in)
        r_out2 = self.linear(r_out[:, -1, :])
        
        return F.log_softmax(r_out2, dim=1)
    
#     def predict(loader, net):
#         predictions = []
#         with torch.no_grad():
#             for data in loader:
#                 inputs = data[0].cuda(0)
#                 labels = data[1].cuda(0)
#                 outputs = net(inputs)
#                 _, predicted = torch.max(outputs.data, 1)
#                 predictions.append(predicted)
#                 total += labels.size(0)
#                 correct += (predicted == labels).sum().item()
#         acc = (correct/total * 100)
#         return predictions
# X.shape - (#batch size, 1, #timepoints, #channels)

**Evaluate function returns values of different criteria like accuracy, precision etc.**
In case you face memory overflow issues, use batch size to control how many samples get evaluated at one time. Use a batch_size that is a factor of length of samples. This ensures that you won't miss any samples.

In [631]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100
    
    predicted = []
    
    for i in range(len(X)//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = Variable(torch.from_numpy(X))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted , multi_class="ovr"))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted), average='macro'))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted) , average='macro'))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted) , average='macro')
            recall = recall_score(Y, np.round(predicted) , average='macro')
            results.append(2*precision*recall/ (precision+recall))
    return results

**Generate random data**
    
*Data format:*

Datatype - float32 (both X and Y)

X.shape - (#samples, 1, #timepoints, #channels)

Y.shape - (#samples)

In [632]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()### Scale the data

In [633]:
def learning_rate_finder():
    model = EEGNet()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-7, weight_decay=1e-2)
    lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
    lr_finder.range_test(train_loader, end_lr=100, num_iter=100, step_mode="exp")
    lr_finder.plot()

In [634]:
def get_accuracy(loader, net):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            inputs = data[0].cuda(0)
            labels = data[1].cuda(0)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = (correct/total * 100)
    return acc

In [635]:
def predict(loader, net):
    correct = 0
    total = 0
    predictions = []
    with torch.no_grad():
        for data in loader:
            inputs = data[0].cuda(0)
            labels = data[1].cuda(0)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            predictions.append(predicted.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = (correct/total * 100)
    return np.concatenate(predictions)

In [636]:
def train_network(X_train, train_loader, num_classes, model, n_epochs,  verbose=False,):
    
    if model == 'EEGNet':
        net = EEGNet(num_classes, False).cuda(0)
    else: 
        net = Combine(num_classes, True).cuda(0)
    
    optimizer = optim.Adam(net.parameters(), lr = 0.001)

    preds_list = []
    labels_list = []
    for epoch in range(n_epochs):
        total_loss = 0
        total_correct = 0

        for batch in train_loader:
        #Get Batch

            inputs = batch[0].cuda(0)
            labels = batch[1].cuda(0)
            
            preds = net(inputs) #Pass batch
        #     criterion=nn.BCEWithLogitsLoss()
            loss = F.cross_entropy(preds, labels.long()) #calculate loss

            optimizer.zero_grad()
            loss.backward()  #calculate gradients
            optimizer.step() #update weights


            preds_list.append(preds)
            labels_list.append(labels)
            total_loss += loss.item()
            total_correct += get_num_correct(preds, labels)
            
        if verbose == True:             
    #     Validation accuracy
    #     params = ["acc", "fmeasure"]
    #     print (params)
    #     print ("Train - ", evaluate(net, X_train, y_train_new, params))
            print("epoch: {0}, Accuracy {1}, loss: {2}".format(epoch, total_correct/len(X_train), loss))
        
    return net

In [637]:
from ipynb.fs.full.evaluation import *
data = load_file("examples/User_1_sampled_annotated_EEG.pickle")
data['inputs'][0].shape

(60, 8)

In [638]:
data['inputs'].shape

(1397, 60, 8)

### Training and testing

In [639]:
# sampled_file = "examples/User_1_sampled_annotated_EEG.pickle"
# with open(sampled_file, 'rb') as handle:
#     dt = pickle.load(handle)

# labels = ["attention", "interest", "effort"]


# results = []
# for label in labels:
    
#     X = dt["inputs"]
#     y = dt[label]

#     #Convert the categories into labels 
#     le = LabelEncoder()
#     y =  le.fit_transform(y)

#     #Train/test split
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

#     # scale the data 
#     scaler = StandardScaler()
#     X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
#     X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

#     #Convert to 4D 
#     X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1],X_train.shape[2])
#     X_test = X_test.reshape(X_test.shape[0],1, X_test.shape[1],X_test.shape[2])

#     #Create train and test loader
#     train = data_utils.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train).long())
#     test = data_utils.TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
#     train_loader = data_utils.DataLoader(train, batch_size=50, shuffle=True)
#     test_loader = data_utils.DataLoader(test, batch_size=50, shuffle=False)
    
#     # train the network
#     net = train_network(X_train, train_loader, 'CNN', verbose=False)
    
#     #get results
#     Results = namedtuple("Results","label train_acc test_acc user")
#     train_acc = get_accuracy(train_loader, test_loader, "train", net)
#     test_acc = get_accuracy(train_loader, test_loader, "test", net)
#     print("{0} results on User 1\ntrain_acc: {1}\tsample_size: {2}\ntest_acc: {3}\tsample_size: {4}\n".format(label, 
#                                                                                            train_acc,  len(X_train),
#                                                                                             test_acc, len(X_test)))
#     results.append(Results(label, train_acc, test_acc,1))

# results = pd.DataFrame(results)
# results.to_csv("results/EEGNet/user1_results_sampled_annotated.csv", index=False)
# results

In [640]:
def standardise(X_train, X_valid):
    # standardize per channel
    means = X_train.mean(axis=(0,2), keepdims=True)
    stds = X_train.std(axis=(0,2), keepdims=True)
    X_train = (X_train - means) / (stds)
    X_valid = (X_valid - means) / (stds)
    return X_train, X_valid


In [641]:
def average(lst): 
    return sum(lst) / len(lst)

In [642]:
def is_slice_in_list(s,l):
    """
    Function for checking whether a slice is in a list. Mainly used for checking whether the 
    """
    len_s = len(s) #so we don't recompute length of s on every iteration
    return any(s == l[i:len_s+i] for i in range(len(l) - len_s+1))

In [643]:
def categorise(y_train, y_valid):
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_valid = le.transform(y_valid)
    return y_train, y_valid, le

In [644]:
def save_plots(model_type, y_true, y_pred, user, label, n_epochs, model, eval_type ):
    if model_type == 'clf':
        # plot confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        saved_file = "results/CNN/clf/confusion/k fold/{3}/{4}/User_{0}_{1}_epochs_{2}.png".format(user,label,n_epochs,model, eval_type)
        plot_confusion_matrix(cm, set(y_true), saved_file ,normalize=True)
        
        #plot model
    if model_type == 'reg':
        saved_file = "results/CNN/reg/y vs y_pred/{3}/{4}/User_{0}_{1}_epochs_{2}.png".format(user,label, n_epochs,model,eval_type)
        plot_model(y_true, y_pred, user, label,file=saved_file)

In [645]:
def get_loaders(X_train, X_valid, y_train, y_valid):
    
    #Convert to 4D 
    X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1],X_train.shape[2])
    X_valid = X_valid.reshape(X_valid.shape[0],1, X_valid.shape[1],X_valid.shape[2])

    # Create train and valid loader
    train = data_utils.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train).long())
    valid = data_utils.TensorDataset(torch.Tensor(X_valid), torch.Tensor(y_valid))
    train_loader = data_utils.DataLoader(train, batch_size=50, shuffle=True)
    valid_loader = data_utils.DataLoader(valid, batch_size=50, shuffle=False)       
    return train_loader, valid_loader

In [646]:
def kfold_predict(X,y, model_type, model, n_epochs):
    """
    Method for running 5 fold cross validation based on a given array of tests
    """
    kf= KFold(n_splits = 5, shuffle = True, random_state =  1)
    
    if model_type == 'clf':
        results = {"Accuracy":[], "Precision":[], "Recall":[], "F1 Score Macro":[],
              "F1 Score Micro":[],"Balanced Accuracy":[]}
    else:
        results = {'RMSE':[], 'R2':[]}
        
    total_predictions = []
    total_true = []
    num_classes = 0
    accuracy = []
    fold = 0
    for train_index, test_index in kf.split(X):
        
        #Train/valid split
        X_train, X_valid = np.concatenate(X[train_index]), np.concatenate(X[test_index])
        y_train, y_valid = np.concatenate(y[train_index]).astype('int'), np.concatenate(y[test_index]).astype('int')
        size = len(X_train) + len(X_valid)
        
        # check the the classes in the validation set, if there are not in training set then skip
        y_valid_classes = list(set(y_valid))
        y_train_classes = list(set(y_train))
        if is_slice_in_list(y_valid_classes, y_train_classes) == False: continue
        
        #standardise per channel
        X_train, X_valid = standardise(X_train, X_valid)
        
        #label the categorical variables
        y_train, y_valid, le = categorise(y_train, y_valid)
        
        #get loaders
        train_loader, valid_loader = get_loaders(X_train, X_valid, y_train, y_valid)
        
         # count the number of classes
        if len(set(y_train)) > num_classes:
            num_classes = len(set(y_train))
       
        # train the network
        time_start = time.time()
        net = train_network(X_train, train_loader, num_classes, model, n_epochs, verbose=False)
        fold += 1 
        print('Fold 1 completed {0}! Time elapsed: {1} seconds'.format(fold, time.time()-time_start))
        
        # make predictions
        y_pred = le.inverse_transform(predict(valid_loader, net))
        y_true = le.inverse_transform(y_valid)
     
        #save total predictions and get results
        total_predictions.append(y_pred)
        total_true.append(y_true) 
        r = get_results(y_true, y_pred, model_type) #returns a dictionary of results   
        valid_acc = get_accuracy(valid_loader, net)
        for key in r: # loop through dictionary to add to all the scores to the results dictionary
            results[key].append(r[key])
        accuracy.append(valid_acc)

    for key in results: # average out the results 
        results[key] = average(results[key])
    accuracy = average(accuracy)    
        
    return results, np.concatenate(total_predictions), np.concatenate(total_true) , num_classes , size , accuracy




#get results
#         Results = namedtuple("Results","label train_acc test_acc user")
#         train_acc = get_accuracy(train_loader, test_loader, "train", net)
#         test_acc = get_accuracy(train_loader, test_loader, "test", net)
#         print("{0} results on User 1\ntrain_acc: {1}\tsample_size: {2}\ntest_acc: {3}\tsample_size: {4}\n".format(label, 
#                                                                                                train_acc,  len(X_train),
#                                                                                                 test_acc, len(X_test)))
#         results.append(Results(label, train_acc, test_acc,1))

In [ ]:
def run_per_user(model):

    time_original = time.time()

    labels = ["attention", "interest", "effort"]

    n_epochs = 100

    window_size_samples = 120
    saved_file = "/cs/home/ybk1/Dissertation/data/saved user and test data/all_users_sampled_{0}_window_annotated_EEG_no_agg.pickle".format(window_size_samples)
    all_tests = load_file(saved_file)
    users = list(all_tests.keys())
    model_type = 'clf'
    results = []
    eval_type = 'per user'
    
    for user in users:
        print("Working on user {0}".format(user))

        for label in labels:

            time_start = time.time()
            dt = all_tests[user] # dictionary of all the individual tests per user

            X = np.array([np.array(x).astype(np.float32) for x in dt['inputs']]) # array of all the inputs for each test
            y = np.array([np.array(x) for x in dt[label]]) #Convert the categories into labels

            # K fold predict 
            r, y_pred, y_true, num_classes, size, accuracy = kfold_predict(X,y,model_type, model, n_epochs)
   
            # get results and add them to the list
            duration = time.time() - time_start
            results.append(collate_results(r, user, label, duration, num_classes, size, model_type, n_epochs, window_size_samples))

            #Save plots
            save_plots(model_type, y_true, y_pred, user, label, n_epochs, model, eval_type )

            print("Finished analysis on User {0}_{1}".format(user,label))
        print("Finished analysis on User {0}".format(user))
    results  = pd.DataFrame(results).to_csv("results/CNN/{3}/tabulated/k fold/{2}/{2}_performance_window_size_{0}_{1}_{4}.csv".format(window_size_samples, n_epochs, model, model_type, eval_type), index=False )
    final_duration = time.time()- time_original
    print("All analyses are complete! Time elapsed: {0}".format(final_duration))
    return results
        
    
def run_cross_user(model):
    time_original = time.time()

    labels = ["attention", "interest", "effort"]
    results = [] # save all results in this list
    
    n_epochs = 10
    window_size_samples = 120
    saved_file = "/cs/home/ybk1/Dissertation/data/saved user and test data/all_users_sampled_{0}_window_annotated_EEG_agg.pickle".format(window_size_samples)
    all_tests_agg = load_file(saved_file)
    users = all_tests_agg.keys()
    model_type = 'clf'
    user ='all'
    eval_type = 'cross user'
   

    for label in labels:
        print("Working on label {0}".format(label))
        time_start = time.time()

        # Store each user in a list to prepare for cross-user analysis
        X = np.array([all_tests_agg[user]['inputs'].astype(np.float32) for user in all_tests_agg])
        y = np.array([all_tests_agg[user][label] for user in all_tests_agg])  

        # train and make predictions
        r, y_pred, y_true, num_classes, size, clf = kfold_predict(X,y,model_type, model, n_epochs)
        print(r['Accuracy'])

         # get results
        duration = time.time() - time_start
        results.append(collate_results(r, user, label, duration, num_classes, size, model_type, n_epochs, window_size_samples))
        
        #Save plots
        save_plots(model_type, y_true, y_pred, user, label, n_epochs, model, eval_type )
                 
        print("Finished analysis on label {0}. Time elapsed {1}".format(label, time.time))
    print("Finished analysis on User {0}".format(user))
    results  = pd.DataFrame(results).to_csv("results/CNN/{3}/tabulated/k fold/{2}/{2}_performance_window_size_{0}_{1}_{4}.csv".format(window_size_samples, n_epochs, model, model_type, eval_type), index=False )
    final_duration = time.time()- time_original
    print("All analyses are complete! Time elapsed: {0}".format(final_duration))

        
run_cross_user('EEGNet')
        
        
        
        
        
        
        
        
#         X = dt["inputs"]
#         y = dt[label]

#         #Convert the categories into labels 
#         le = LabelEncoder()
#         y =  le.fit_transform(y)

#         #Train/test split
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

#         # scale the data 
#         scaler = StandardScaler()
#         X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
#         X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

#         #Convert to 4D 
#         X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1],X_train.shape[2])
#         X_test = X_test.reshape(X_test.shape[0],1, X_test.shape[1],X_test.shape[2])

#         #Create train and test loader
#         train = data_utils.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train).long())
#         test = data_utils.TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
#         train_loader = data_utils.DataLoader(train, batch_size=50, shuffle=True)
#         test_loader = data_utils.DataLoader(test, batch_size=50, shuffle=False)

#         # train the network
#         net = train_network(X_train, train_loader, 'CNN', verbose=False)

#         #get results
#         Results = namedtuple("Results","label train_acc test_acc user")
#         train_acc = get_accuracy(train_loader, test_loader, "train", net)
#         test_acc = get_accuracy(train_loader, test_loader, "test", net)
#         print("{0} results on User 1\ntrain_acc: {1}\tsample_size: {2}\ntest_acc: {3}\tsample_size: {4}\n".format(label, 
#                                                                                                train_acc,  len(X_train),
#                                                                                                 test_acc, len(X_test)))
#         results.append(Results(label, train_acc, test_acc,1))

# results = pd.DataFrame(results)
# results.to_csv("results/EEGNet/user1_results_sampled_annotated.csv", index=False)
# # results

Working on label attention
Training fold 1! Time elapsed: 16.307141304016113 seconds
Training fold 2! Time elapsed: 17.350783824920654 seconds
Training fold 3! Time elapsed: 17.204396724700928 seconds
Training fold 4! Time elapsed: 20.78155541419983 seconds
Training fold 5! Time elapsed: 18.60366916656494 seconds
0.2226906831115162
Normalized confusion matrix
[[0.03231821 0.23741454 0.28589186 0.29148539 0.15288999]
 [0.02735828 0.26460932 0.2959072  0.27161304 0.14051215]
 [0.04978277 0.28222303 0.32041999 0.23805214 0.10952209]
 [0.03598732 0.26626888 0.33078501 0.25191124 0.11504755]
 [0.03336951 0.28594683 0.36597938 0.21134021 0.10336408]]
Finished analysis on label attention
Working on label interest


**Run**